In [1]:
pip install -q kaggle

In [2]:
# upload kaggle.json
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mrifqisyatria","key":"06a9b30418327f9bd931f23c0638d1ef"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

kaggle.json


In [4]:

# test kaggle dataset list
!kaggle datasets list

ref                                                             title                                            size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  ----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
akshaydattatraykhare/diabetes-dataset                           Diabetes Dataset                                  9KB  2022-10-06 08:55:25          10765        345  1.0              
whenamancodes/covid-19-coronavirus-pandemic-dataset             COVID -19 Coronavirus Pandemic Dataset           11KB  2022-09-30 04:05:11           8554        277  1.0              
stetsondone/video-game-sales-by-genre                           Video Game Sales by Genre                        12KB  2022-10-31 17:56:01            708         23  1.0              
whenamancodes/credit-card-customers-prediction                  Credit Card Cust

In [55]:
!wget --no-check-certificate \
  https://storage.googleapis.com/dataset-uploader/bbc/bbc-text.csv \
  -O /tmp/bbc-text.csv

--2022-11-09 11:42:20--  https://storage.googleapis.com/dataset-uploader/bbc/bbc-text.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.197.128, 74.125.142.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5057493 (4.8M) [text/csv]
Saving to: ‘/tmp/bbc-text.csv’

/tmp/bbc-text.csv   100%[===================>]   4.82M  --.-KB/s    in 0.03s   

2022-11-09 11:42:20 (150 MB/s) - ‘/tmp/bbc-text.csv’ saved [5057493/5057493]



In [57]:
import pandas as pd
df = pd.read_csv('/tmp/bbc-text.csv')
df

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
...,...,...
2220,business,cars pull down us retail figures us retail sal...
2221,politics,kilroy unveils immigration policy ex-chatshow ...
2222,entertainment,rem announce new glasgow concert us band rem h...
2223,politics,how political squabbles snowball it s become c...


In [48]:
df = pd.read_csv('reviewArchives/Women Dresses Reviews Dataset .csv', sep='\t')

In [58]:
df.head(9)

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
5,politics,howard hits back at mongrel jibe michael howar...
6,politics,blair prepares to name poll date tony blair is...
7,sport,henman hopes ended in dubai third seed tim hen...
8,sport,wilkinson fit to face edinburgh england captai...


In [60]:
df.columns

Index(['category', 'text'], dtype='object')

In [63]:
import nltk, os, re, string

from keras.layers import Input, LSTM, Bidirectional, SpatialDropout1D, Dropout, Flatten, Dense, Embedding, BatchNormalization
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical


from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn

nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [64]:
# data category one-hot-encoding
category = pd.get_dummies(df.category)
df_with_category = pd.concat([df, category], axis=1)
df_with_category = df_with_category.drop(columns='category')
df_with_category.head(10)

,text,business,entertainment,politics,sport,tech
0,tv future in the hands of viewers with home th...,0,0,0,0,1
1,worldcom boss left books alone former worldc...,1,0,0,0,0
2,tigers wary of farrell gamble leicester say ...,0,0,0,1,0
3,yeading face newcastle in fa cup premiership s...,0,0,0,1,0
4,ocean s twelve raids box office ocean s twelve...,0,1,0,0,0
5,howard hits back at mongrel jibe michael howar...,0,0,1,0,0
6,blair prepares to name poll date tony blair is...,0,0,1,0,0
7,henman hopes ended in dubai third seed tim hen...,0,0,0,1,0
8,wilkinson fit to face edinburgh england captai...,0,0,0,1,0
9,last star wars not for children the sixth an...,0,1,0,0,0


In [67]:
df_with_category.text = df_with_category.text.apply(lambda x: x.lower())


def cleaner(data):
    return(data.translate(str.maketrans('','', string.punctuation)))
    
    df_with_category.text = df_with_category.text.apply(lambda x: lem(x))


lemmatizer = WordNetLemmatizer()
def lem(data):
    pos_dict = {'N': wn.NOUN, 'V': wn.VERB, 'J': wn.ADJ, 'R': wn.ADV}
    return(' '.join([lemmatizer.lemmatize(w,pos_dict.get(t, wn.NOUN)) for w,t in nltk.pos_tag(data.split())]))
    df_new.title = df_new.title.apply(lambda x: lem(x))
    df_new.content = df_new.content.apply(lambda x: lem(x))


def rem_numbers(data):
    return re.sub('[0-9]+','',data)
    df_with_category['text'].apply(rem_numbers)
    




In [68]:
# total data
df_with_category.head(10)

,text,business,entertainment,politics,sport,tech
0,tv future in the hands of viewers with home th...,0,0,0,0,1
1,worldcom boss left books alone former worldc...,1,0,0,0,0
2,tigers wary of farrell gamble leicester say ...,0,0,0,1,0
3,yeading face newcastle in fa cup premiership s...,0,0,0,1,0
4,ocean s twelve raids box office ocean s twelve...,0,1,0,0,0
5,howard hits back at mongrel jibe michael howar...,0,0,1,0,0
6,blair prepares to name poll date tony blair is...,0,0,1,0,0
7,henman hopes ended in dubai third seed tim hen...,0,0,0,1,0
8,wilkinson fit to face edinburgh england captai...,0,0,0,1,0
9,last star wars not for children the sixth an...,0,1,0,0,0


In [69]:
text = df_with_category['text'].values
labeling = df_with_category[['business', 'entertainment', 'politics', 'sport', 'tech']].values

In [70]:
text

array(['tv future in the hands of viewers with home theatre systems  plasma high-definition tvs  and digital video recorders moving into the living room  the way people watch tv will be radically different in five years  time.  that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes. with the us leading the trend  programmes and other content will be delivered to viewers via home networks  through cable  satellite  telecoms companies  and broadband service providers to front rooms and portable devices.  one of the most talked-about technologies of ces has been digital and personal video recorders (dvr and pvr). these set-top boxes  like the us s tivo and the uk s sky+ system  allow people to record  store  play  pause and forward wind tv programmes when they want.  essentially  the technology allows for much more personalised tv. they are also being built-in 

In [71]:

labeling

array([[0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0],
       ...,
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0]], dtype=uint8)

In [72]:

from sklearn.model_selection import train_test_split
text_train, text_test, labeling_train, labeling_test = train_test_split(text, labeling, test_size=0.2, shuffle=True)

In [74]:

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
tokenizer = Tokenizer(num_words=5000, oov_token='x', filters='!"#$%&()*+,-./:;<=>@[\]^_`{|}~ ')
tokenizer.fit_on_texts(text_train) 
tokenizer.fit_on_texts(text_test)
 
train_seq = tokenizer.texts_to_sequences(text_train)
test_seq = tokenizer.texts_to_sequences(text_test)
 
train_pad = pad_sequences(train_seq) 
test_pad = pad_sequences(test_seq)

In [75]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=64),
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy',)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          320000    
                                                                 
 lstm (LSTM)                 (None, 128)               98816     
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 5)                 645       
                                                                 
Total params: 435,973
Trainable params: 435,973
Non-trainable params: 0
_________________________________________________________________


In [76]:

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.9 and logs.get('val_accuracy')>0.9):
      self.model.stop_training = True
      print("\n akurasi dah sampe 90% lebih nih")
callbacks = myCallback()

In [78]:
# model fit
history = model.fit(train_pad, labeling_train, epochs=66, 
                    validation_data=(test_pad, labeling_test), verbose=2, callbacks=[callbacks], validation_steps=30)

Epoch 1/66
56/56 - 18s - loss: 1.5933 - accuracy: 0.2556 - val_loss: 1.5520 - val_accuracy: 0.3303 - 18s/epoch - 323ms/step
Epoch 2/66
56/56 - 11s - loss: 1.2460 - accuracy: 0.4382 - val_loss: 1.0404 - val_accuracy: 0.5146 - 11s/epoch - 188ms/step
Epoch 3/66
56/56 - 11s - loss: 0.9011 - accuracy: 0.6511 - val_loss: 1.1164 - val_accuracy: 0.6022 - 11s/epoch - 188ms/step
Epoch 4/66
56/56 - 11s - loss: 0.5278 - accuracy: 0.7921 - val_loss: 0.6517 - val_accuracy: 0.7798 - 11s/epoch - 194ms/step
Epoch 5/66
56/56 - 10s - loss: 0.2916 - accuracy: 0.8702 - val_loss: 0.6253 - val_accuracy: 0.8022 - 10s/epoch - 186ms/step
Epoch 6/66
56/56 - 10s - loss: 0.4642 - accuracy: 0.8596 - val_loss: 0.7552 - val_accuracy: 0.7393 - 10s/epoch - 186ms/step
Epoch 7/66
56/56 - 12s - loss: 0.1155 - accuracy: 0.9685 - val_loss: 0.6691 - val_accuracy: 0.7910 - 12s/epoch - 221ms/step
Epoch 8/66
56/56 - 10s - loss: 0.0564 - accuracy: 0.9865 - val_loss: 0.5710 - val_accuracy: 0.8472 - 10s/epoch - 187ms/step
Epoch 9/